# Case study of August 20, 2022

In [1]:
%store  -r city_center clipped_upwind_wedge clipped_downwind_wedge stats

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point

In [17]:
tracked_cells = xr.open_dataset('C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/stats/trackstats_20220801.0000_20220831.2359.nc')
tracked_cells

<xarray.Dataset>
Dimensions:                  (tracks: 11705, times: 100)
Coordinates:
  * tracks                   (tracks) int64 0 1 2 3 ... 11701 11702 11703 11704
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) int32 ...
    base_time                (tracks, times) datetime64[ns] ...
    meanlat                  (tracks, times) float32 ...
    meanlon                  (tracks, times) float32 ...
    area                     (tracks, times) float32 ...
    cloudnumber              (tracks, times) float64 ...
    ...                       ...
    start_split_tracknumber  (tracks) float64 ...
    start_split_timeindex    (tracks) float64 ...
    start_split_cloudnumber  (tracks) float64 ...
    end_merge_tracknumber    (tracks) float64 ...
    end_merge_timeindex      (tracks) float64 ...
    end_merge_cloudnumber    (tracks) float64 ...
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Tue Aug  1 20:43:12 2023
    startdate:             20220801.0000
    enddate:               20220831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

In [18]:
# Extract all the start_basetimes
tracked_cells.start_basetime.values

array(['2022-08-01T00:33:45.023000064', '2022-08-01T00:48:14.157999872',
       '2022-08-01T01:08:19.980999936', ...,
       '2022-08-31T22:57:30.164999936', '2022-08-31T23:03:33.900999936',
       '2022-08-31T23:20:42.420999936'], dtype='datetime64[ns]')

# Download and gid radars close to the base_times

## Extract time range

In [83]:
# Define your time range
start_time = np.datetime64('2022-08-20T19:03:00')
end_time = np.datetime64('2022-08-20T19:03:59')

# Create a mask where base_time is within the specified range
time_mask = (tracked_cells.start_basetime >= start_time) & (tracked_cells.start_basetime <= end_time)

# Apply this mask across the 'tracks' dimension
filtered_ds = tracked_cells.where(time_mask, drop=True)
filtered_ds

<xarray.Dataset>
Dimensions:                  (tracks: 6, times: 100)
Coordinates:
  * tracks                   (tracks) int64 6403 6404 6405 6406 6407 6408
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) float64 15.0 3.0 5.0 2.0 11.0 2.0
    base_time                (tracks, times) datetime64[ns] 2022-08-20T19:03:...
    meanlat                  (tracks, times) float32 29.53 29.56 ... nan nan
    meanlon                  (tracks, times) float32 -95.85 -95.84 ... nan nan
    area                     (tracks, times) float32 46.0 58.25 57.0 ... nan nan
    cloudnumber              (tracks, times) float64 12.0 11.0 9.0 ... nan nan
    ...                       ...
    start_split_tracknumber  (tracks) float64 6.252e+03 6.317e+03 ... nan nan
    start_split_timeindex    (tracks) float64 25.0 14.0 10.0 nan nan nan
    start_split_cloudnumber  (tracks) float64 2.0 4.0 9.0 nan nan nan
    end_merge_tracknumber    (tracks) float64 nan nan nan nan nan nan
    end_merge_timeindex      (tracks) float64 nan nan nan nan nan nan
    end_merge_cloudnumber    (tracks) float64 nan nan nan nan nan nan
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Tue Aug  1 20:43:12 2023
    startdate:             20220801.0000
    enddate:               20220831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

## Extract storms in the region

### Urban area

In [84]:
# Initiation at urban area
def lat_lon_to_cartesian(lat, lon, R=6371):
    x = R * np.radians(lon)
    y = R * np.radians(lat)
    return x, y

start_lon = filtered_ds['cell_meanlon'].isel(times=0)
start_lat = filtered_ds['cell_meanlat'].isel(times=0)
start_hour = filtered_ds['start_basetime'].dt.hour
hour_bin = np.arange(0, 25, 1)

center_lat, center_lon, radius = city_center[1], city_center[0], 76.35
center_x, center_y = lat_lon_to_cartesian(center_lat, center_lon)
storm_x, storm_y = lat_lon_to_cartesian(start_lat, start_lon)
distances = np.sqrt((storm_x - center_x)**2 + (storm_y - center_y)**2)
storms_in_circle = start_hour[distances <= radius]
hist_storms_in_circle, bins = np.histogram(storms_in_circle, bins=hour_bin, range=(0, 24), density=False)
hist_storms_in_circle_LT = np.roll(hist_storms_in_circle, -6)
storms_in_circle
# Fraction of E.SDC tracks to all tracks
#hist_storms_in_circle_LT_frac = 100 * (hist_storms_in_circle_LT / hist_starthour_LT)

<xarray.DataArray 'hour' (tracks: 2)>
array([19, 19], dtype=int64)
Coordinates:
  * tracks   (tracks) int64 6403 6406
    times    int64 0
Attributes:
    long_name:  Start Epoch time of each track

### Upwind area

In [85]:
storm_points = [Point(lon, lat) for lon, lat in zip(start_lon, start_lat)]
storms_in_upwind_wedge = [start_hour[i] for i, point in enumerate(storm_points) if clipped_upwind_wedge.contains(point)]
storms_in_upwind_wedge = xr.concat(storms_in_upwind_wedge, dim='tracks')
hist_storms_in_upwind_wedge, bins = np.histogram(storms_in_upwind_wedge, bins=hour_bin, range=(0, 24), density=False)
storms_in_upwind_wedge

<xarray.DataArray 'hour' (tracks: 2)>
array([19, 19], dtype=int64)
Coordinates:
  * tracks   (tracks) int64 6404 6405
Attributes:
    long_name:  Start Epoch time of each track

### Downwind area

In [86]:
storm_points = [Point(lon, lat) for lon, lat in zip(start_lon, start_lat)]
storms_in_downwind_wedge = [start_hour[i] for i, point in enumerate(storm_points) if clipped_downwind_wedge.contains(point)]
storms_in_downwind_wedge = xr.concat(storms_in_downwind_wedge, dim='tracks')
hist_storms_in_downwind_wedge, bins = np.histogram(storms_in_downwind_wedge, bins=hour_bin, range=(0, 24), density=False)
storms_in_downwind_wedge

ValueError: must supply at least one object to concatenate

## Filter the stats data

In [73]:
circle = storms_in_circle.tracks.values
stats_circle_case_1 = filtered_ds.sel(tracks=circle)

upwind = storms_in_upwind_wedge.tracks.values
stats_upwind_case_1 = filtered_ds.sel(tracks=upwind)

downwind = storms_in_downwind_wedge.tracks.values
stats_downwind_case_1 = filtered_ds.sel(tracks=downwind)

## Calculate cfad

In [74]:
tracks_upwind = stats_upwind_case_1.tracks.values
tracks_urban = stats_circle_case_1.tracks.values
tracks_downwind = stats_downwind_case_1.tracks.values

In [17]:
start_upwind = [9703, 9704, 9707, 9709, 9720, 9719, 9718, 9726, 9725, 9724, 9722, 9732, 9729, 9730, 9735, 9733, 9739, 9736, 9741, 9745, 9746, 9748, 9749, 9753, 9758, 9762, 9760, 9764, 9769, 9770, 9775, 9772, 9778, 9779, 9785, 9788, 9789, 9791, 9795, 9798, 9792, 9794, 9798, 9800, 9802, 9804, 9809, 9815, 9817, 9820, 9822, 9834, 9836, 9844, 9846, 9848, 9861, 9858, 9855, 9866, 9868, 9874, 9875, 9879, 9876, 9877, 9885, 9887, 9888, 9889, 9894, 9900, 9902, 9905, 9906, 9908, 9909, 9913, 9915, 9918, 9920, 9921, 9924, 9925, 9926, 9929, 9930, 9931, 9935, 9935, 9936, 9938, 9941, 9942, 9943, 9952, 9957, 9958, 9959, 9962, 9966, 9967, 9969, 9970, 9972, 9976, 9982, 9987, 9990, 9988, 9993, 9998, 10000, 10001, 9997, 10003, 10004, 10002, 10011, 10019, 10018, 10021, 10026, 10034, 10030, 10031, 10042, 10037, 10046, 10047, 10056, 10053, 10051, 10057, 10054, 10058, 10059, 10063, 10065, 10067, 10069, 10068, 10074, 10077, 10078, 10079, 10081, 10084, 10085]
end_upwind = [9703, 9704, 9707, 9719, 9718, 9724, 9722, 9729, 9730, 9735, 9733, 9736, 9741, 9745, 9753, 9746, 9748, 9749, 9753, 9758, 9762, 9760, 9764, 9769, 9770, 9772, 9778, 9779, 9785, 9788, 9789, 9791, 9798, 9800, 9815, 9817, 9820, 9822, 9834, 9844, 9846, 9858, 9855, 9868, 9874, 9875, 9876, 9877, 9885, 9887, 9888, 9889, 9900, 9902, 9905, 9906, 9908, 9909, 9913, 9915, 9920, 9921, 9924, 9925, 9926, 9929, 9930, 9931, 9935, 9935, 9936, 9938, 9941, 9942, 9943, 9952, 9957, 9958, 9959, 9962, 9966, 9969, 9970, 9976, 9982, 9987, 9990, 9993, 9998, 10000, 10001, 10004, 10002, 10011, 10018, 10021, 10026, 10030, 10031, 10037, 10046, 10047, 10053, 10051, 10057, 10054, 10058, 10059, 10063, 10065, 10067, 10069, 10068, 10074, 10077, 10078, 10079, 10081, 10084, 10086]
start_urban = [9712, 9713, 9714, 9711, 9715, 9723, 9731, 9739, 9747, 9754, 9750, 9752, 9761, 9768, 9769, 9773, 9776, 9777, 9781, 9783, 9786, 9787, 9790, 9805, 9806, 9807, 9811, 9813, 9818, 9823, 9824, 9826, 9827, 9831, 9832, 9833, 9835, 9841, 9842, 9845, 9849, 9853, 9854, 9857, 9867, 9870, 9871, 9882, 9884, 9897, 9899, 9901, 9917, 9937, 9961, 9968, 9975, 9992, 9994, 10005, 10006, 10016, 10023, 10024, 10036, 10049, 10055, 10060, 10061, 10062, 10064, 10076, 10075, 10082, 10080]
end_urban = [9712, 9713, 9714, 9711, 9715, 9726, 9723, 9731, 9747, 9750, 9752, 9761, 9768, 9769, 9773, 9776, 9777, 9781, 9783, 9786, 9787, 9790, 9795, 9805, 9806, 9807, 9811, 9811, 9813, 9818, 9823, 9824, 9826, 9827, 9831, 9832, 9833, 9836, 9835,  9841, 9842, 9845, 9848, 9849, 9853, 9854, 9861, 9857, 9867, 9870, 9871, 9882, 9884, 9894, 9897, 9899, 9901, 9917, 9937, 9961, 9968, 9975, 9992, 9994, 9997, 10005, 10006, 10016, 10022, 10023, 10024, 10034, 10036, 10049, 10055, 10060, 10061, 10062, 10064, 10076, 10075, 10082, 10080]
start_downwind = [9751, 9903, 9983]
end_downwind = [9741, 9742, 9751, 9754, 9866, 9903, 9910, 9980, 9983]



In [18]:
filtered_upwind = filtered_ds.sel(tracks=start_upwind)
filtered_urban = filtered_ds.sel(tracks=start_urban)

In [19]:
filtered_urban

<xarray.Dataset>
Dimensions:                  (tracks: 75, times: 100)
Coordinates:
  * tracks                   (tracks) int64 9712 9713 9714 ... 10075 10082 10080
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) float64 12.0 4.0 5.0 1.0 ... 2.0 3.0 2.0
    base_time                (tracks, times) datetime64[ns] 2021-06-28T12:14:...
    meanlat                  (tracks, times) float32 29.88 29.9 ... nan nan
    meanlon                  (tracks, times) float32 -94.95 -94.97 ... nan nan
    area                     (tracks, times) float32 39.75 41.75 ... nan nan
    cloudnumber              (tracks, times) float64 29.0 29.0 29.0 ... nan nan
    ...                       ...
    start_split_tracknumber  (tracks) float64 nan nan nan ... nan nan nan
    start_split_timeindex    (tracks) float64 nan nan nan 11.0 ... nan nan nan
    start_split_cloudnumber  (tracks) float64 nan nan nan 17.0 ... nan nan nan
    end_merge_tracknumber    (tracks) float64 nan 9.613e+03 ... nan nan
    end_merge_timeindex      (tracks) float64 nan 19.0 20.0 nan ... nan nan nan
    end_merge_cloudnumber    (tracks) float64 nan 1.0 1.0 nan ... nan nan nan
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5

In [87]:
%store -r a

In [89]:
ds1 = a
ds2 = filtered_ds

In [90]:
ds2.cell_meanlat

<xarray.DataArray 'cell_meanlat' (tracks: 6, times: 100)>
array([[29.525297, 29.558691, 29.58405 , 29.631126, 29.67589 , 29.701641,
        29.729458, 29.767027, 29.807041, 29.86155 , 29.906157, 29.930647,
        29.954082, 29.97252 , 29.98969 ,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan],
       [29.009853, 29.025917, 29.037966,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
...
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan],
       [28.946566, 28.978575,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,       nan]], dtype=float32)
Coordinates:
  * tracks   (tracks) int64 6403 6404 6405 6406 6407 6408
  * times    (times) int64 0 1 2 3 4 5 6 7 8 9 ... 90 91 92 93 94 95 96 97 98 99
Attributes:
    long_name:  Mean latitude of a convective cell
    units:      degrees_north

In [91]:
ds1.lat

<xarray.DataArray 'lat' (y: 401, x: 401)>
array([[28.5686618 , 28.56870087, 28.56873974, ..., 28.56873974,
        28.56870087, 28.5686618 ],
       [28.57315805, 28.57319713, 28.57323601, ..., 28.57323601,
        28.57319713, 28.57315805],
       [28.57765431, 28.57769339, 28.57773227, ..., 28.57773227,
        28.57769339, 28.57765431],
       ...,
       [30.35816875, 30.35820925, 30.35824956, ..., 30.35824956,
        30.35820925, 30.35816875],
       [30.36266499, 30.3627055 , 30.3627458 , ..., 30.3627458 ,
        30.3627055 , 30.36266499],
       [30.36716122, 30.36720174, 30.36724205, ..., 30.36724205,
        30.36720174, 30.36716122]])
Coordinates:
    lat      (y, x) float64 28.57 28.57 28.57 28.57 ... 30.37 30.37 30.37 30.37
    lon      (y, x) float64 -96.1 -96.1 -96.09 -96.09 ... -94.05 -94.04 -94.04
  * y        (y) float64 -1e+05 -9.95e+04 -9.9e+04 ... 9.9e+04 9.95e+04 1e+05
  * x        (x) float64 -1e+05 -9.95e+04 -9.9e+04 ... 9.9e+04 9.95e+04 1e+05
Attributes:
    long_name:      latitude of grid cell center
    units:          degree_N
    standard_name:  Latitude

In [92]:
import numpy as np
import xarray as xr

# Function to find the nearest index in an array for a given value
def find_nearest_index(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return np.unravel_index(idx, array.shape)

# Convert lat and lon in ds1 to NumPy arrays for processing
lat_np = ds1.lat.values
lon_np = ds1.lon.values

# Flatten the cell_meanlat and cell_meanlon arrays from ds2
flat_cell_meanlat = ds2.cell_meanlat.values.flatten()
flat_cell_meanlon = ds2.cell_meanlon.values.flatten()

# Initialize a mask for ds1 with False values
mask = np.full(ds1.lat.shape, False)

# Iterate over the flattened cell_meanlat and cell_meanlon arrays
for meanlat, meanlon in zip(flat_cell_meanlat, flat_cell_meanlon):
    if not np.isnan(meanlat) and not np.isnan(meanlon):
        # Find the nearest index in the lat and lon arrays
        idx_lat, idx_lon = find_nearest_index(lat_np, meanlat), find_nearest_index(lon_np, meanlon)

        # Update the mask
        mask[idx_lat, idx_lon] = True

# Convert the mask to an xarray DataArray
mask_da = xr.DataArray(mask, coords=ds1.lat.coords, dims=ds1.lat.dims)

# Filter the first dataset using the mask DataArray
filtered_ds1 = ds1.where(mask_da, drop=True)

# filtered_ds1 now contains only the data corresponding to cell_meanlat and cell_meanlon in ds2


In [93]:
filtered_ds1

<xarray.Dataset>
Dimensions:                       (time: 1, z: 41, y: 65, x: 57)
Coordinates:
  * time                          (time) object 2022-08-20 19:03:40.525000
  * z                             (z) float64 0.0 500.0 1e+03 ... 1.95e+04 2e+04
    lat                           (y, x) float64 28.57 28.57 ... 29.99 29.99
    lon                           (y, x) float64 -96.1 -96.01 ... -94.12 -94.04
  * y                             (y) float64 -1e+05 -9.75e+04 ... 5.8e+04
  * x                             (x) float64 -1e+05 -9.05e+04 ... 1e+05
Data variables:
    reflectivity                  (time, z, y, x) float32 nan nan ... nan nan
    velocity                      (time, z, y, x) float32 nan nan ... nan nan
    differential_phase            (time, z, y, x) float32 nan nan ... nan nan
    spectrum_width                (time, z, y, x) float32 nan nan ... nan nan
    clutter_filter_power_removed  (time, z, y, x) float32 nan nan ... nan nan
    cross_correlation_ratio       (time, z, y, x) float32 nan nan ... nan nan
    kdp                           (time, z, y, x) float64 nan nan ... nan nan
    differential_reflectivity     (time, z, y, x) float32 nan nan ... nan nan
    ROI                           (time, z, y, x) float32 nan nan ... nan nan

In [94]:
import numpy as np
import xarray as xr

# Assuming 'reflectivity' is the variable name in your dataset
reflectivity = filtered_ds1['differential_reflectivity']

# Define the mask condition, for example, mask out values below 20 dBZ
mask_condition = np.isnan(reflectivity)

# Apply the mask to create a masked array
masked_reflectivity = np.ma.masked_where(mask_condition, reflectivity.values)

# Get the dimension names of the original 'reflectivity' variable
dims = reflectivity.dims

# Replace the 'reflectivity' variable in the dataset with the masked array, providing dimension names
filtered_ds1['reflectivity'] = (dims, masked_reflectivity)

# Now 'reflectivity' in ds1 is replaced with the masked array where values below 20 dBZ are masked


In [95]:
import numpy as np
import xarray as xr

# Assuming 'reflectivity' is loaded from your dataset and is an xarray DataArray
reflectivity = filtered_ds1['differential_reflectivity']

# First, get the numpy array from the DataArray
reflectivity_np = reflectivity.values

# Define the mask condition (masking NaN values here as an example)
mask_condition = np.isnan(reflectivity_np)

# Apply the mask to create a masked array
masked_reflectivity_np = np.ma.masked_where(mask_condition, reflectivity_np)

# Now, squeeze the numpy array to remove the singleton dimension
# This operation reduces the shape from (1, 41, 401, 401) to (41, 401, 401)
masked_reflectivity_squeezed = np.squeeze(masked_reflectivity_np)

# Ensure the final shape is what you expect
assert masked_reflectivity_squeezed.shape == (41, 401, 401)

# If you need to work with this array within the xarray framework,
# you might want to convert it back to an xarray DataArray.
# You should specify the correct dimensions if they are known and applicable.
dims = reflectivity.dims[1:]  # Assuming the first dimension is the one to remove

# Create a new DataArray. This step is optional and depends on whether you need
# the result as an xarray DataArray for further operations within xarray.
masked_reflectivity = xr.DataArray(masked_reflectivity_squeezed, dims=dims, coords={dim: reflectivity.coords[dim] for dim in dims})

# Now, masked_reflectivity is ready and has the shape (41, 401, 401).


AssertionError: 

In [96]:
import numpy as np
import xarray as xr

# Assuming 'reflectivity' is your xarray.DataArray from the dataset
reflectivity = filtered_ds1['differential_reflectivity']

# Extract the numpy array from the DataArray
reflectivity_np = reflectivity.values

# Define the mask condition. Here, masking NaN values as an example.
# Adjust this condition to fit your specific needs, such as masking values below a threshold.
mask_condition = np.isnan(reflectivity_np)

# Apply the mask to create a masked array
masked_reflectivity_np = np.ma.masked_where(mask_condition, reflectivity_np)

# Squeeze the masked array to remove the singleton dimension
# This directly addresses your requirement to change the shape to (41, 401, 401)
masked_reflectivity = np.squeeze(masked_reflectivity_np)

# Verify the shape is as expected
print(masked_reflectivity.shape)  # This should print (41, 401, 401)

# At this point, masked_reflectivity is a NumPy masked array with the shape you wanted.


(41, 65, 57)


In [64]:
reflectivity.shape

(1, 41, 65, 57)

In [97]:
masked_reflectivity

masked_array(
  data=[[[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        ...,

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --,

In [63]:
%store masked_reflectivity

Stored 'masked_reflectivity' (MaskedArray)


In [32]:
filtered_ds1['reflectivity']

<xarray.DataArray 'reflectivity' (time: 1, z: 41, y: 401, x: 401)>
array([[[[ 0.01277933,  0.05783932,  0.06709348, ...,         nan,
                  nan,         nan],
         [-0.05911751,         nan,         nan, ...,         nan,
                  nan,         nan],
         [-0.10423183,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,  2.3076048 ],
         [        nan,         nan,         nan, ...,         nan,
                  nan,  2.2861733 ],
         [ 0.4966761 ,         nan,  0.50925434, ...,         nan,
                  nan,  1.370781  ]],

        [[-0.0024256 ,  0.04254729,  0.06013403, ...,         nan,
                  nan,         nan],
         [-0.02827725,         nan,         nan, ...,         nan,
                  nan,         nan],
         [-0.04934229,         nan,         nan, ...,         nan,
                  nan,         nan],
...
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]]]], dtype=float32)
Coordinates:
  * time     (time) object 2022-08-20 19:03:40.525000
  * z        (z) float64 0.0 500.0 1e+03 1.5e+03 ... 1.9e+04 1.95e+04 2e+04
    lat      (y, x) float64 28.57 28.57 28.57 28.57 ... 30.37 30.37 30.37 30.37
    lon      (y, x) float64 -96.1 -96.1 -96.09 -96.09 ... -94.05 -94.04 -94.04
  * y        (y) float64 -1e+05 -9.95e+04 -9.9e+04 ... 9.9e+04 9.95e+04 1e+05
  * x        (x) float64 -1e+05 -9.95e+04 -9.9e+04 ... 9.9e+04 9.95e+04 1e+05

In [21]:
longitudes = filtered_urban.cell_meanlon.isel(times=0).values
latitudes = filtered_urban.cell_meanlat.isel(times=0).values
start_times = filtered_urban.start_basetime.values


In [ ]:
longitudes

array(-94.947815, dtype=float32)

In [22]:


# Calculate minimum and maximum values
min_longitude = longitudes.min()
max_longitude = longitudes.max()
min_latitude = latitudes.min()
max_latitude = latitudes.max()

# Print the results
print(f"Minimum Longitude: {min_longitude}")
print(f"Maximum Longitude: {max_longitude}")
print(f"Minimum Latitude: {min_latitude}")
print(f"Maximum Latitude: {max_latitude}")


Minimum Longitude: -95.77266693115234
Maximum Longitude: -94.29417419433594
Minimum Latitude: 29.29880142211914
Maximum Latitude: 30.344860076904297


In [23]:
import pandas as pd
data = {
    'Longitude': longitudes,
    'Latitude': latitudes,
    'Start_Basetime': pd.to_datetime(start_times, unit='s')  # Adjust the unit if needed
}
track_df = pd.DataFrame(data)


In [24]:
sorted_df = track_df.sort_values(by=['Longitude', 'Latitude'])
sorted_df

,Longitude,Latitude,Start_Basetime
6,-95.772667,29.470287,2021-06-28 12:40:13.078000128
28,-95.725662,29.760731,2021-06-28 14:08:03.931000064
17,-95.722542,29.591759,2021-06-28 13:24:34.464999936
70,-95.707909,29.384020,2021-06-28 19:26:29.128999936
67,-95.629341,29.393852,2021-06-28 19:26:29.128999936
...,...,...,...
26,-94.967613,29.753098,2021-06-28 14:01:49.309999872
0,-94.947815,29.880405,2021-06-28 12:14:47.825999872
33,-94.938187,29.867575,2021-06-28 14:20:30.691000064
14,-94.309380,29.776140,2021-06-28 13:12:02.932999936


In [ ]:
n

In [ ]:
pd.unique(start_times)

array(['2021-06-28T12:02:04.199000064', '2021-06-28T12:08:26.116000000',
       '2021-06-28T12:27:30.409999872', '2021-06-28T12:33:51.180000000',
       '2021-06-28T12:40:13.078000128', '2021-06-28T12:46:35.048000000',
       '2021-06-28T12:52:56.644000000', '2021-06-28T12:59:18.374000128',
       '2021-06-28T13:05:40.112000000', '2021-06-28T13:12:02.932999936',
       '2021-06-28T13:24:34.464999936', '2021-06-28T13:18:18.740999936',
       '2021-06-28T13:37:06.254000128', '2021-06-28T13:43:20.892999936',
       '2021-06-28T13:49:37.536999936', '2021-06-28T13:55:42.597999872',
       '2021-06-28T14:08:03.931000064', '2021-06-28T14:20:30.691000064',
       '2021-06-28T14:26:31.291000064', '2021-06-28T14:32:41.292999936',
       '2021-06-28T14:38:46.947000064', '2021-06-28T14:50:57.449999872',
       '2021-06-28T14:44:56.843000064', '2021-06-28T14:57:19.363000064',
       '2021-06-28T15:03:41.123000064', '2021-06-28T15:09:41.992999936',
       '2021-06-28T15:15:59.691000064', '2021-06-28

In [ ]:
'''
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box, Point, Polygon

# Function to create an extended wedge-shaped polygon from the circle edge
def create_extended_wedge(center, inner_radius, outer_radius, angle_start, angle_end, num_points=100):
    angle_start_rad = np.radians(angle_start)
    angle_end_rad = np.radians(angle_end)
    inner_angles = np.linspace(angle_start_rad, angle_end_rad, num_points)
    outer_angles = np.linspace(angle_end_rad, angle_start_rad, num_points)
    inner_points = [(center[0] + inner_radius * np.cos(a), center[1] + inner_radius * np.sin(a)) for a in inner_angles]
    outer_points = [(center[0] + outer_radius * np.cos(a), center[1] + outer_radius * np.sin(a)) for a in outer_angles]
    return Polygon(inner_points + outer_points + [inner_points[0]])

# Load the shapefile
shapefile_path = "C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/shapefile/2022_Shapefile/2022_Developed_Shapefile.shp"
gdf = gpd.read_file(shapefile_path)

# Get the largest shape by area
largest_shape = gdf.iloc[gdf['geometry'].area.idxmax()]

# Extract the individual shapes from the MultiPolygon
shapes = list(largest_shape['geometry'].geoms)

# List to store the results
shape_data = []

# For each individual shape, compute the enclosing circle, area, and centroid
for shape in shapes:
    enclosing_circle = shape.convex_hull.minimum_rotated_rectangle.boundary.minimum_rotated_rectangle.buffer(0.01).boundary
    enclosing_circle_center = Point(enclosing_circle.centroid.x, enclosing_circle.centroid.y)
    enclosing_circle_radius = enclosing_circle.distance(enclosing_circle_center)
    shape_area = shape.area
    shape_centroid = shape.centroid
    shape_data.append({
        "enclosing_circle_center": (enclosing_circle_center.x, enclosing_circle_center.y),
        "enclosing_circle_radius": enclosing_circle_radius,
        "shape_area": shape_area,
        "shape_centroid": (shape_centroid.x, shape_centroid.y)
    })

# Identify the shape with the largest enclosing circle
largest_enclosing_circle_data = max(shape_data, key=lambda x: x["enclosing_circle_radius"])

# Using the data from the largest enclosing circle obtained previously:
city_center = largest_enclosing_circle_data["enclosing_circle_center"]
urban_radius = largest_enclosing_circle_data["enclosing_circle_radius"]

# Updated definitions
downwind_start = 100
downwind_end = 100
sector_angle = 160  # in degrees
mean_wind_angle = 125.37  # in degrees

# Adjust the scaling factors based on the new urban radius
downwind_start_scaled = downwind_start * urban_radius / 25
downwind_end_scaled = downwind_end * urban_radius / 25

# Define the bounding box for clipping
minx, miny, maxx, maxy = -96.3, 29, -93.9, 31
bounding_box = box(minx, miny, maxx, maxy)

# Create the urban circle as a polygon
urban_circle_poly = Point(city_center).buffer(urban_radius)

# Create the extended wedges
downwind_wedge_poly = create_extended_wedge(city_center, urban_radius, downwind_end_scaled, mean_wind_angle-(sector_angle/2), mean_wind_angle+(sector_angle/2))
upwind_wedge_poly = create_extended_wedge(city_center, urban_radius, downwind_end_scaled, mean_wind_angle+180-(sector_angle/2), mean_wind_angle+180+(sector_angle/2))
right_no_impact_wedge_poly = create_extended_wedge(city_center, urban_radius, downwind_end_scaled, mean_wind_angle-180+(sector_angle/2), mean_wind_angle-(sector_angle/2))
left_no_impact_wedge_poly = create_extended_wedge(city_center, urban_radius, downwind_end_scaled, mean_wind_angle+(sector_angle/2), mean_wind_angle+180-(sector_angle/2))

# Clip the regions to the bounding box
clipped_urban_circle = urban_circle_poly.intersection(bounding_box)
clipped_downwind_wedge = downwind_wedge_poly.intersection(bounding_box)
clipped_upwind_wedge = upwind_wedge_poly.intersection(bounding_box)
clipped_right_no_impact_wedge = right_no_impact_wedge_poly.intersection(bounding_box)
clipped_left_no_impact_wedge = left_no_impact_wedge_poly.intersection(bounding_box)

# Function to check if a point is inside any of the clipped regions
def is_point_inside_clipped_regions(lon, lat, clipped_regions):
    point = Point(lon, lat)
    return any(point.within(region) for region in clipped_regions)

# Function to plot a clipped region
def plot_clipped_region(ax, clipped_region, color):
    if not clipped_region.is_empty:
        x, y = clipped_region.exterior.xy
        ax.fill(x, y, color=color, alpha=0.5)

# Function to plot geographic evolution of a group of tracks with starting point
def plot_geographic_evolution_with_start(ax, group_tracks, stats_dataset, color, group_number, clipped_regions):
    # Extract data for the selected track
    track_data = stats_dataset.sel(tracks=group_tracks)

    # Check if the track data is not empty
    if track_data and track_data.cell_meanlon.size > 0 and track_data.cell_meanlat.size > 0:
        # Check if the starting point is within any of the clipped regions
        if is_point_inside_clipped_regions(track_data.cell_meanlon.values[0], track_data.cell_meanlat.values[0], clipped_regions):
            # Latitude and Longitude Evolution
            ax.plot(track_data.cell_meanlon.values, track_data.cell_meanlat.values, marker='o', color=color, linestyle='-', linewidth=1.5, label=f'Group {group_number}: {group_tracks}')
            # Mark the starting point
            ax.plot(track_data.cell_meanlon.values[0], track_data.cell_meanlat.values[0], marker='X', color=color, markersize=12)
        else:
            print(f"Track ID: {group_tracks} is outside the clipped regions.")
    else:
        print(f"No data found for track ID: {group_tracks}")

# Function to create the plot for each pair of tracks
def plot_tracks(groups, plot_number, stats_dataset):
    fig, axs = plt.subplots(figsize=(10, 8))

    colors = ['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'orange']
    clipped_regions = [clipped_urban_circle, clipped_downwind_wedge, clipped_upwind_wedge, clipped_right_no_impact_wedge, clipped_left_no_impact_wedge]
    
    for i, group_tracks in enumerate(groups):
        plot_geographic_evolution_with_start(axs, group_tracks, stats_dataset, colors[i % len(colors)], i+1, clipped_regions)

    plot_clipped_region(axs, clipped_urban_circle, 'grey')
    plot_clipped_region(axs, clipped_downwind_wedge, 'blue')
    plot_clipped_region(axs, clipped_upwind_wedge, 'red')
    plot_clipped_region(axs, clipped_right_no_impact_wedge, 'green')
    plot_clipped_region(axs, clipped_left_no_impact_wedge, 'orange')

    axs.set_title(f'Geographic Evolution of Convective Cells - Plot {plot_number}', fontsize=16)
    axs.set_xlabel('Longitude', fontsize=14)
    axs.set_ylabel('Latitude', fontsize=14)
    axs.grid(True)
    axs.set_axisbelow(True)
    axs.legend(loc='upper right')
    plt.show()

# Loop to create plots for each pair of tracks
stats_dataset = stats # Load your dataset with track information here

# Assuming filtered_ds.tracks.values is an array-like structure of track IDs
track_values = filtered_ds.tracks.values

# Determine the size of each group
group_size = 7  # You can adjust this size as needed

# Split the track_values into groups of the specified size
for i in range(0, len(track_values), group_size):
    group_tracks = track_values[i:i + group_size]
    plot_tracks(group_tracks, i // group_size, stats_dataset)

'''

In [ ]:
stats

<xarray.Dataset>
Dimensions:                  (tracks: 54932, times: 100)
Coordinates:
  * tracks                   (tracks) int64 0 1 2 3 ... 345280 345281 345282
  * times                    (times) int64 0 1 2 3 4 5 6 ... 94 95 96 97 98 99
Data variables: (12/37)
    track_duration           (tracks) int32 11 2 4 6 3 2 2 32 ... 3 2 2 4 1 3 8
    base_time                (tracks, times) datetime64[ns] 2021-06-01T00:00:...
    meanlat                  (tracks, times) float32 29.12 29.13 ... nan nan
    meanlon                  (tracks, times) float32 -95.23 -95.25 ... nan nan
    area                     (tracks, times) float32 22.0 17.0 16.0 ... nan nan
    cloudnumber              (tracks, times) float64 1.0 2.0 4.0 ... nan nan nan
    ...                       ...
    start_split_tracknumber  (tracks) float64 nan nan nan ... 1.167e+04 nan nan
    start_split_timeindex    (tracks) float64 nan nan nan nan ... 16.0 nan nan
    start_split_cloudnumber  (tracks) float64 nan nan nan nan ... 1.0 nan nan
    end_merge_tracknumber    (tracks) float64 nan nan nan nan ... nan nan nan
    end_merge_timeindex      (tracks) float64 nan nan nan nan ... nan nan nan
    end_merge_cloudnumber    (tracks) float64 nan nan nan nan ... nan nan nan
Attributes:
    Title:                 Statistics of each track
    Institution:           Pacific Northwest National Laboratory
    Contact:               Zhe Feng, zhe.feng@pnnl.gov
    Created_on:            Sat Sep 16 01:48:11 2023
    startdate:             20210601.0000
    enddate:               20210831.2359
    timegap_hour:          0.25
    time_resolution_hour:  0.0833
    pixel_radius_km:       0.5